In [1]:
pip install pandas requests



  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.2 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/11.6 MB 1.2 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.6 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.6 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/11.6 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/11.6 MB 1.1 MB/s eta 0:00:10
   ------- -------------------------------- 2.1/11.6 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.6 MB 1.1 MB/s eta 0:00:09
   -------- ----------------------------

In [2]:
pip install --upgrade TikTokApi

  Using cached tiktokapi-7.1.0-py3-none-any.whl.metadata (9.7 kB)
  Using cached playwright-1.52.0-py3-none-win_amd64.whl.metadata (3.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pyee-13.0.0-py3-none-any.whl.metadata (2.9 kB)
Using cached tiktokapi-7.1.0-py3-none-any.whl (55 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached playwright-1.52.0-py3-none-win_amd64.whl (34.8 MB)
Using cached pyee-13.0.0-py3-none-any.whl (15 kB)

   ---- ----------------------------------- 1/9 [pyee]
   -------- ------------------------------- 2/9 [h11]
   -------- ------------------------------- 2/9 [h11]
   ----------

In [4]:
!pip install --upgrade TikTokApi playwright pandas nest_asyncio
!playwright install

  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Attempting uninstall: nest_asyncio
    Found existing installation: nest-asyncio 1.5.6
    Uninstalling nest-asyncio-1.5.6:
      Successfully uninstalled nest-asyncio-1.5.6


In [5]:
!pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)


In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
# Configuration
load_dotenv()  # Load environment variables from .env file if it exists

RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")
if not RAPIDAPI_KEY:
    raise ValueError("RAPIDAPI_KEY is not set in the environment variables.")

RAPIDAPI_HOST = "tiktok-api23.p.rapidapi.com"
BASE_URL = "https://tiktok-api23.p.rapidapi.com/api/post/trending"


#Creaate 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

def fetch_multiple_batches(max_results=500):
    """Fetch multiple batches to get more than 8 results"""
    all_items = []
    cursor = "0"

    while len(all_items) < max_results:
        headers = {
            "X-RapidAPI-Host": RAPIDAPI_HOST,
            "X-RapidAPI-Key": RAPIDAPI_KEY
        }

        params = {
            "count": 8,
            "cursor": cursor
        }

        try:
            response = requests.get(BASE_URL, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            if not data.get('itemList'):
                break

            all_items.extend(data['itemList'])
            cursor = data.get('cursor', "0")

            # Stop if no more items available
            if not data.get('hasMore', False):
                break

        except Exception as e:
            print(f"Error fetching batch: {e}")
            break

    return all_items[:max_results]

def process_data(items):
    processed = []
    for item in items:
        processed.append({
            "video_id": item.get('id'),
            "author": item.get('author', {}).get('uniqueId'),
            "description": item.get('desc'),
            "likes": item.get('stats', {}).get('diggCount'),
            "comments": item.get('stats', {}).get('commentCount'),
            "shares": item.get('stats', {}).get('shareCount'),
            "plays": item.get('stats', {}).get('playCount'),
            "hashtags": ", ".join([h['title'] for h in item.get('challenges', [])]),
            "music": item.get('music', {}).get('title'),
            "create_time": pd.to_datetime(item.get('createTime'), unit='s') if item.get('createTime') else None,
            "video_url": f"https://www.tiktok.com/@{item.get('author', {}).get('uniqueId')}/video/{item.get('id')}"
        })
    return pd.DataFrame(processed)

# Fetch data
print("Fetching trending TikTok videos...")
items = fetch_multiple_batches(max_results=500)
df = process_data(items)

print(f"\nSuccessfully fetched {len(df)} videos")
display(df.head(10))

# Save to CSV inside 'data' folder
output_path = os.path.join('data', 'tiktok_trending30_elvis.csv')
df.to_csv(output_path, index=False)
print(f"\nData saved to '{output_path}'")

Fetching trending TikTok videos...

Successfully fetched 500 videos


,video_id,author,description,likes,comments,shares,plays,hashtags,music,create_time,video_url
0,7506333786653707525,zayynelly,2-11 is the worst shift ever ain't no debate t...,90900,2437,7493,910100,"fyp, foryou, explorerpage✨, zayynelly, job",original sound,2025-05-20 01:28:58,https://www.tiktok.com/@zayynelly/video/750633...
1,7506568036363635975,pokrov,все расскажу в тгк : аня pokrov,143900,8506,8619,3000000,,оригинальный звук,2025-05-20 16:37:52,https://www.tiktok.com/@pokrov/video/750656803...
2,7506109122748648735,eleanor06011,I thought no one would show up for my 45th…but...,67400,34400,847,1600000,"single, divoced, singleladies, singlelife, bir...",Very Sad,2025-05-19 10:57:50,https://www.tiktok.com/@eleanor06011/video/750...
3,7499176235516431638,lunicoragazzoanni60baby,never give up,3600000,15000,253500,53300000,,sonido original,2025-04-30 18:33:55,https://www.tiktok.com/@lunicoragazzoanni60bab...
4,7506666880808275246,awtuuhm,Idkk where the name come from but it was good😂...,114300,751,4055,444700,"food, fy",original sound,2025-05-20 23:01:53,https://www.tiktok.com/@awtuuhm/video/75066668...
5,7505795680141724958,logan.pastor,This isn’t even all of it😄 #foryoupage #colleg...,118100,274,8369,561000,"foryoupage, collegemoveout, postgradlife, move...",original sound,2025-05-18 14:41:09,https://www.tiktok.com/@logan.pastor/video/750...
6,7506473348491136298,ieuehhrdhhehe,#cops #deafman #tase #fyp #police #usa #viral ...,28800,47,296,807600,"cops, deafman, tase, fyp, police, usa, viral, ...",原聲 - ieuehhrdhhehe,2025-05-20 12:00:00,https://www.tiktok.com/@ieuehhrdhhehe/video/75...
7,7506720213157629214,treinadorkaka,@Treinador Kaká 🙏🏻🙏🏻💪🏻💪🏻,353800,7383,236500,5200000,,original sound,2025-05-21 02:28:30,https://www.tiktok.com/@treinadorkaka/video/75...
8,7506743554643922219,julia_..marie,#cheer #sport,95300,10800,2656,642000,"cheer, sport",Bunna Summa,2025-05-21 03:59:07,https://www.tiktok.com/@julia_..marie/video/75...
9,7501860734247210270,weluvu.josh,like ion need no more 😭#fyp #viral #foryou #fy...,542300,9406,206600,3500000,"fyp, viral, foryou, fypシ, relatable, fy, fypシ゚...",Make you Fall in Love,2025-05-08 00:11:21,https://www.tiktok.com/@weluvu.josh/video/7501...



Data saved to 'data\tiktok_trending30_elvis.csv'
